<a href="https://colab.research.google.com/github/SkinHeadDressSom/Ultimate-X-ray-AI/blob/main/CXR_split_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 16.0 MB/s eta 0:00:00


In [16]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import pydicom
from PIL import Image
import cv2
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool

## making Yolo format

In [41]:
csv_path = "/content/drive/MyDrive/UltimateXray/ultimateXray.csv"
output_dir = "/content/drive/MyDrive/UltimateXray/preprocess"

image_output_dir = os.path.join(output_dir, "images")
label_output_dir = os.path.join(output_dir, "labels")

os.makedirs(image_output_dir, exist_ok=True)
os.makedirs(label_output_dir, exist_ok=True)

df = pd.read_csv(csv_path)
df

,image_id,class_name,x_min,y_min,x_max,y_max,file_type
0,/content/drive/MyDrive/UltimateXray/VinBigData...,No Finding,NaN,NaN,NaN,NaN,dicom
1,/content/drive/MyDrive/UltimateXray/VinBigData...,No Finding,NaN,NaN,NaN,NaN,dicom
2,/content/drive/MyDrive/UltimateXray/VinBigData...,No Finding,NaN,NaN,NaN,NaN,dicom
3,/content/drive/MyDrive/UltimateXray/VinBigData...,No Finding,NaN,NaN,NaN,NaN,dicom
4,/content/drive/MyDrive/UltimateXray/VinBigData...,No Finding,NaN,NaN,NaN,NaN,dicom
...,...,...,...,...,...,...,...
50504,/content/drive/MyDrive/UltimateXray/VinBigData...,Pleural effusion,240.0,1550.0,562.0,2001.0,dicom
50505,/content/drive/MyDrive/UltimateXray/VinBigData...,Pulmonary fibrosis,1163.0,787.0,1338.0,941.0,dicom
50506,/content/drive/MyDrive/UltimateXray/VinBigData...,ILD,299.0,664.0,794.0,1508.0,dicom
50507,/content/drive/MyDrive/UltimateXray/VinBigData...,Other lesion,6.0,670.0,272.0,1736.0,dicom


In [42]:
class_mapping = {name: idx for idx, name in enumerate(df['class_name'].unique())}
df['class_id'] = df['class_name'].map(class_mapping)
df

,image_id,class_name,x_min,y_min,x_max,y_max,file_type,class_id
0,/content/drive/MyDrive/UltimateXray/VinBigData...,No Finding,NaN,NaN,NaN,NaN,dicom,0
1,/content/drive/MyDrive/UltimateXray/VinBigData...,No Finding,NaN,NaN,NaN,NaN,dicom,0
2,/content/drive/MyDrive/UltimateXray/VinBigData...,No Finding,NaN,NaN,NaN,NaN,dicom,0
3,/content/drive/MyDrive/UltimateXray/VinBigData...,No Finding,NaN,NaN,NaN,NaN,dicom,0
4,/content/drive/MyDrive/UltimateXray/VinBigData...,No Finding,NaN,NaN,NaN,NaN,dicom,0
...,...,...,...,...,...,...,...,...
50504,/content/drive/MyDrive/UltimateXray/VinBigData...,Pleural effusion,240.0,1550.0,562.0,2001.0,dicom,17
50505,/content/drive/MyDrive/UltimateXray/VinBigData...,Pulmonary fibrosis,1163.0,787.0,1338.0,941.0,dicom,13
50506,/content/drive/MyDrive/UltimateXray/VinBigData...,ILD,299.0,664.0,794.0,1508.0,dicom,11
50507,/content/drive/MyDrive/UltimateXray/VinBigData...,Other lesion,6.0,670.0,272.0,1736.0,dicom,15


In [43]:
df['file_exists'] = df['image_id'].apply(lambda x: os.path.exists(x))
df = df[df['file_exists']].drop(columns=['file_exists'])

In [44]:
def convert_dicom_to_png_and_resize(dicom_path, png_path):
    dicom = pydicom.dcmread(dicom_path)
    image = dicom.pixel_array
    scaled_image = ((image - np.min(image)) / (np.max(image) - np.min(image)) * 255).astype(np.uint8)
    resized_image = Image.fromarray(scaled_image).resize((640, 640))
    resized_image.save(png_path)

In [45]:
def ensure_image_validity(file_path, file_type, image_save_path):
    try:
        with Image.open(image_save_path) as img:
            img.verify()
    except Exception:
        if file_type in ['dicom', 'dcm']:
            convert_dicom_to_png_and_resize(file_path, image_save_path)
        else:
            img = cv2.imread(file_path)
            if img is None:
                raise ValueError("Invalid image file.")
            img_resized = cv2.resize(img, (640, 640))
            cv2.imwrite(image_save_path, img_resized)

In [46]:
def process_row(row):
    file_path = row['image_id']
    file_type = row['file_type']

    # Prepare output paths
    image_name = os.path.splitext(os.path.basename(file_path))[0]
    image_save_path = os.path.join(image_output_dir, f"{image_name}.png")
    label_save_path = os.path.join(label_output_dir, f"{image_name}.txt")

    try:
        ensure_image_validity(file_path, file_type, image_save_path)

        img = cv2.imread(image_save_path)
        if img is None:
            raise ValueError("Invalid image file.")

        original_height, original_width = img.shape[:2]

        # Calculate the scaling factors
        width_scale = 640 / original_width
        height_scale = 640 / original_height

        # Scale bounding box coordinates
        width = (row['x_max'] - row['x_min']) * width_scale
        height = (row['y_max'] - row['y_min']) * height_scale
        x_center = (row['x_min'] + row['x_max']) / 2 * width_scale
        y_center = (row['y_min'] + row['y_max']) / 2 * height_scale

        # Write YOLO label to file
        label = f"{row['class_id']} {x_center / 640:.6f} {y_center / 640:.6f} {width / 640:.6f} {height / 640:.6f}\n"
        with open(label_save_path, 'a') as f:
            f.write(label)

    except Exception as e:
        # Cleanup invalid files
        if os.path.exists(image_save_path):
            os.remove(image_save_path)
        if os.path.exists(label_save_path):
            os.remove(label_save_path)

In [47]:
def parallel_process(df):
    args = [row for _, row in df.iterrows()]
    with Pool() as pool:
        list(tqdm(pool.imap(process_row, args), total=len(args)))

In [48]:
parallel_process(df)

100%|██████████| 49241/49241 [57:07<00:00, 14.37it/s]


In [49]:
print(len(os.listdir(os.path.join(image_output_dir))))
print(len(os.listdir(os.path.join(label_output_dir))))

13985
13985


## train val test split

In [50]:
import os
import random
import shutil
from pathlib import Path

In [51]:
images_dir = "/content/drive/MyDrive/UltimateXray/preprocess/images"
labels_dir = "/content/drive/MyDrive/UltimateXray/preprocess/labels"
base_dir = "/content/drive/MyDrive/UltimateXray/YOLO"

subdirs = {
    "train": {"images": f"{base_dir}/images/train", "labels": f"{base_dir}/labels/train"},
    "val": {"images": f"{base_dir}/images/val", "labels": f"{base_dir}/labels/val"},
    "test": {"images": f"{base_dir}/images/test", "labels": f"{base_dir}/labels/test"},
}

for split, paths in subdirs.items():
    os.makedirs(paths["images"], exist_ok=True)
    os.makedirs(paths["labels"], exist_ok=True)

In [52]:
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.png')])

labeled_images = []
unlabeled_images = []

In [53]:
for image in image_files:
    label_path = os.path.join(labels_dir, Path(image).stem + ".txt")
    if os.path.exists(label_path):
        labeled_images.append(image)
    else:
        unlabeled_images.append(image)

In [54]:
random.seed(42)
random.shuffle(labeled_images)
random.shuffle(unlabeled_images)

In [55]:
def split_data(data, train_ratio, val_ratio):
    train_size = int(len(data) * train_ratio)
    val_size = int(len(data) * val_ratio)
    train = data[:train_size]
    val = data[train_size:train_size + val_size]
    test = data[train_size + val_size:]
    return train, val, test

train_labeled, val_labeled, test_labeled = split_data(labeled_images, train_ratio, val_ratio)
train_unlabeled, val_unlabeled, test_unlabeled = split_data(unlabeled_images, train_ratio, val_ratio)

In [56]:
train_set = train_labeled + train_unlabeled
val_set = val_labeled + val_unlabeled
test_set = test_labeled + test_unlabeled

In [57]:
def copy_files(split, images, subdir):
    for image in tqdm(images):
        label_path = os.path.join(labels_dir, Path(image).stem + ".txt")
        image_src = os.path.join(images_dir, image)
        label_dst_dir = subdirs[subdir]["labels"]
        image_dst_dir = subdirs[subdir]["images"]

        # Copy image
        shutil.copy(image_src, image_dst_dir)

        # Copy label if it exists
        if os.path.exists(label_path):
            shutil.copy(label_path, label_dst_dir)

In [58]:
copy_files("train", train_set, "train")
copy_files("val", val_set, "val")
copy_files("test", test_set, "test")

100%|██████████| 1399/1399 [09:12<00:00,  2.53it/s]


In [59]:
class_names = df['class_name'].unique()
class_dict = {idx: name for idx, name in enumerate(class_names)}
class_dict

{0: 'No Finding',
 1: 'Infiltrate',
 2: 'Atelectasis',
 3: 'Pneumonia',
 4: 'Cardiomegaly',
 5: 'Effusion',
 6: 'Pneumothorax',
 7: 'Mass',
 8: 'Nodule',
 9: 'Aortic enlargement',
 10: 'Pleural thickening',
 11: 'ILD',
 12: 'Nodule/Mass',
 13: 'Pulmonary fibrosis',
 14: 'Lung Opacity',
 15: 'Other lesion',
 16: 'Infiltration',
 17: 'Pleural effusion',
 18: 'Calcification',
 19: 'Consolidation'}

In [60]:
import yaml

base_path = "/content/drive/MyDrive/UltimateXray/YOLO"

dataset_config = {
    'path': base_dir,
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'names': class_dict
}

yaml_file_path = os.path.join(base_dir, 'dataset.yaml')

with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(dataset_config, yaml_file, default_flow_style=False)